In [ ]:
import os
os.chdir(os.path.join(os.getcwd(), '../../'))

from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from yfiles_jupyter_graphs import GraphWidget
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Layout

# Neo4j credentials
url = "bolt://0.0.0.0:7687"
username = "neo4j"
password = "neo4j"

# LangChain Neo4j graph
graph = Neo4jGraph(url=url, username=username, password=password)

# Get list of instances
from front_end.instances import list_of_class_instances
instances = list_of_class_instances()
instances = [str(instance).replace('user_owl.', '') for instance in instances]
instances = ['_'.join(instance.split('_')[1:]) for instance in instances]

# Graph display function
def showGraph(query):
    out = widgets.Output(layout={'border': '1px solid black'})
    
    with out:
        session = GraphDatabase.driver(url, auth=(username, password)).session()
        graph_data = session.run(query).graph()
        
        # Create and configure the GraphWidget
        widget = GraphWidget(graph=graph_data, widget_layout = Layout(height='1000px', width='2550'))
        widget.set_sidebar(start_with='Data')
        widget.node_label_mapping = 'display_name'
        widget.edge_label_mapping = 'display_name'
        widget.hierarchic_layout()
        
        # Set fixed default size BEFORE display
        widget.layout = widgets.Layout(width='2550px', height='1000px')

        # Sliders to resize the widget
        width_slider = widgets.FloatSlider(
            value=2550, min=400, max=2550, step=50,
            description='Width:', continuous_update=True
        )
        height_slider = widgets.FloatSlider(
            value=1000, min=400, max=1000, step=50,
            description='Height:', continuous_update=True
        )

        def update_graph_size(change, dimension):
            new_size = change['new']
            if dimension == 'width':
                widget.layout.width = f'{new_size}px'
            elif dimension == 'height':
                widget.layout.height = f'{new_size}px'

        width_slider.observe(lambda change: update_graph_size(change, 'width'), names='value')
        height_slider.observe(lambda change: update_graph_size(change, 'height'), names='value')

        # Display sliders and graph widget
        display(width_slider, height_slider, widget)

    return out

# Create a Tab widget to hold one tab per instance
tab = widgets.Tab()
graph_widgets = []

# Generate graphs and assign tabs
for i, instance in enumerate(instances):
    query = f"""
    MATCH (a)
    WHERE a.display_name = "{instance}"
    MATCH (a)-[r*1..15]->(b)
    RETURN a, r, b
    """
    graph_output = showGraph(query)
    graph_widgets.append(graph_output)

# Assign to tab children and set titles
tab.children = graph_widgets
for i, instance in enumerate(instances):
    tab.set_title(i, instance)

# Display the full tab view
display(tab)


<!-- MATCH (a)
WHERE a.uri in ["AAE","GAN"]
MATCH (a)-[r*1..2]->(b)
RETURN a, r, b -->

<!-- MATCH (a)-[r:ns0__hasTaskType]-(b)
RETURN a, r, b -->


<!-- MATCH (a)
    WHERE a.uri in ["AAE","GAN"]
    MATCH (a)-[r*1..2]->(b)
    RETURN a, r, b


MATCH (n)-[r]->(m) RETURN n, r, m -->

<!-- MATCH (a)-[r:ns0__hasTaskType]->(b)
RETURN a, r, b; -->